<a href="https://colab.research.google.com/github/ramonfercruz/dados_enem/blob/main/predi%C3%A7%C3%A3o_nota_matematica_enem.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
import zipfile
import pandas as pd
import sklearn
from sklearn.model_selection import train_test_split

In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
arquivo = '/content/drive/MyDrive/Dados Enem/microdados_enem_2019.zip'

In [ ]:
colunas = [ 'NU_INSCRICAO','NU_NOTA_CN', 'NU_NOTA_CH', 'NU_NOTA_LC','NU_NOTA_MT','NU_IDADE', 'TP_SEXO', 'SG_UF_RESIDENCIA','Q006', 'Q025', 'TP_COR_RACA','IN_TREINEIRO' , 'TP_PRESENCA_MT','TP_SEXO','TP_ESCOLA', 'TP_ST_CONCLUSAO', 'TP_ANO_CONCLUIU', 'Q001', 'Q002', 'CO_PROVA_MT']

In [ ]:
with zipfile.ZipFile(arquivo) as file_zip:
  with file_zip.open('DADOS/MICRODADOS_ENEM_2019.csv','r') as file_csv:
    df_enem = pd.read_csv(file_csv, sep=';', encoding='latin1',usecols=colunas)

In [ ]:
df_enem = df_enem[(df_enem['TP_PRESENCA_MT'] == 1) & (df_enem['IN_TREINEIRO'] == 0)]

In [ ]:
tipos_dados = dict(df_enem.dtypes)

In [ ]:
for coluna, tipo in tipos_dados.items():
  if tipo == object:
    chaves = dict(df_enem[coluna].value_counts().sort_index()).keys()
    dict_map = {}
    for i, chave in  enumerate(chaves):
      dict_map[chave] = i
    dict_map[None] = -1
    df_enem[coluna] = df_enem[coluna].map(dict_map)
    


In [ ]:
for coluna in list(df_enem.columns):
  df_enem[coluna].fillna(0,inplace=True)

In [ ]:
df_enem.NU_INSCRICAO.count()

3179862

In [ ]:
import numpy as np

# Selecionado um registro aleatório entre os valores de 0 a 10:
semente = np.random.choice(10, 1)
semente
indices = np.arange(0,df_enem.NU_INSCRICAO.count(),semente)


In [ ]:
amostra_2 = df_enem.iloc[list(indices), :]

In [ ]:
amostra = df_enem.sample(frac=0.05)

In [ ]:
amostra.shape

(158993, 19)

In [ ]:
x_amostra_treino, x_amostra_teste = train_test_split(amostra)

In [ ]:
x_amostra_treino.shape

(119244, 19)

In [ ]:
x_amostra_teste.shape

(39749, 19)

In [ ]:
fetuares  = [ 'NU_NOTA_CN', 'NU_NOTA_CH', 'NU_NOTA_LC','NU_IDADE', 'TP_SEXO', 'Q006','TP_ESCOLA','TP_ST_CONCLUSAO']
target = ['NU_NOTA_MT']

In [ ]:
x_treino = x_amostra_treino[fetuares]
y_treino = x_amostra_treino[target]
x_teste = x_amostra_teste[fetuares]
y_teste = x_amostra_teste[target]

In [ ]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()  
x_treino = sc.fit_transform(x_treino)  
x_teste = sc.transform(x_teste)

In [ ]:
from sklearn.ensemble import RandomForestRegressor
regressor = RandomForestRegressor(n_estimators= 100)

In [ ]:
 
%%time
regressor.fit(x_treino, y_treino.to_numpy().ravel())

CPU times: user 1min 10s, sys: 284 ms, total: 1min 10s
Wall time: 1min 11s


RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=100, n_jobs=None, oob_score=False,
                      random_state=None, verbose=0, warm_start=False)

In [ ]:
y_pred_test = regressor.predict(x_teste)
y_pred_train = regressor.predict(x_treino)

In [ ]:
import numpy as np
from sklearn import metrics

In [ ]:
print('MAE:', metrics.mean_absolute_error(y_treino.to_numpy().ravel(), y_pred_train).round(8)  )
print('MSE:', metrics.mean_squared_error(y_treino.to_numpy().ravel(), y_pred_train).round(8) )  
print('RMSE:', np.sqrt(metrics.mean_squared_error(y_treino.to_numpy().ravel(), y_pred_train)).round(8))

MAE: 21.61236819
MSE: 749.96483206
RMSE: 27.38548579


In [ ]:
print('MAE:', metrics.mean_absolute_error(y_teste.to_numpy().ravel(), y_pred_test).round(8))
print('MSE:', metrics.mean_squared_error(y_teste.to_numpy().ravel(), y_pred_test).round(8) )  
print('RMSE:', np.sqrt(metrics.mean_squared_error(y_teste.to_numpy().ravel(), y_pred_test)).round(8))

MAE: 58.40811968
MSE: 5389.58847163
RMSE: 73.41381663


In [ ]:
import pickle

In [ ]:
pickle.dump(regressor, open('/content/drive/MyDrive/Dados Enem/modelo_RandomForestRegressor.sav', 'wb'))

In [ ]:
x_amostra_treino['p'] = y_pred_train
x_amostra_treino['nota_matematica'] = y_treino

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
x_amostra_treino

,NU_INSCRICAO,SG_UF_RESIDENCIA,NU_IDADE,TP_SEXO,TP_COR_RACA,TP_ST_CONCLUSAO,TP_ANO_CONCLUIU,TP_ESCOLA,IN_TREINEIRO,TP_PRESENCA_MT,CO_PROVA_MT,NU_NOTA_CN,NU_NOTA_CH,NU_NOTA_LC,NU_NOTA_MT,Q001,Q002,Q006,Q025,p,nota_matematica
349040,190001353676,1,17,0,1,2,0,3,0,1,516.0,462.9,392.1,505.3,566.0,3,4,3,1,530.038,566.0
4654101,190005658846,4,38,1,1,1,13,1,0,1,515.0,606.9,650.6,676.2,683.4,4,6,5,1,688.461,683.4
935887,190001940535,5,24,1,3,1,4,1,0,1,517.0,595.0,527.1,518.8,568.7,1,1,1,1,556.618,568.7
3373425,190004378117,25,19,1,1,1,2,1,0,1,518.0,535.5,606.9,552.6,770.4,4,4,5,1,695.107,770.4
2084366,190003089028,5,17,0,3,2,0,2,0,1,518.0,480.7,445.8,473.4,445.8,7,2,2,1,449.482,445.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2566677,190003571345,16,21,1,3,1,3,1,0,1,515.0,484.2,534.1,535.8,573.0,7,1,1,0,557.110,573.0
3947580,190004952286,9,24,1,2,1,6,1,0,1,515.0,535.1,526.3,417.0,628.0,1,1,2,0,583.175,628.0
3663433,190004668129,25,53,1,1,2,0,2,0,1,518.0,398.1,361.2,424.5,532.7,2,3,1,1,484.937,532.7
1672573,190002677230,23,19,1,1,2,0,2,0,1,518.0,489.9,567.5,582.7,670.7,7,4,5,1,607.321,670.7
